<a href="https://colab.research.google.com/github/mayssajr/STM32_digit_recognition/blob/main/mnist_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Charger le dataset MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normaliser en [0,1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Ajouter une dimension "channel" (car CNN attend images 28x28x1)
x_train = np.expand_dims(x_train, -1)  # shape: (60000, 28, 28, 1)
x_test  = np.expand_dims(x_test, -1)

print("Shape train:", x_train.shape)
print("Shape test :", x_test.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Shape train: (60000, 28, 28, 1)
Shape test : (10000, 28, 28, 1)


In [ ]:
model = keras.Sequential([
    layers.Conv2D(8, (3,3), activation="relu", input_shape=(28,28,1)),
    layers.Conv2D(16, (3,3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 149,418 (583.66 KB)

 Trainable params: 149,418 (583.66 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.1)

# Évaluer sur test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("\nTest accuracy:", test_acc)


Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8363 - loss: 0.5795 - val_accuracy: 0.9750 - val_loss: 0.0995
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9708 - loss: 0.1002 - val_accuracy: 0.9800 - val_loss: 0.0733
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9823 - loss: 0.0601 - val_accuracy: 0.9845 - val_loss: 0.0589
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9861 - loss: 0.0465 - val_accuracy: 0.9872 - val_loss: 0.0512
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9900 - loss: 0.0323 - val_accuracy: 0.9882 - val_loss: 0.0477
313/313 - 2s - 7ms/step - accuracy: 0.9852 - loss: 0.0444

Test accuracy: 0.9851999878883362


In [ ]:
model.save("mnist_cnn.h5")


In [ ]:
# Conversion en TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Quantization (int8 → beaucoup plus petit)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Sauvegarder
with open("mnist_cnn_final.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmptop2u2rr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135060637267152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060637269840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060637267920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060637267344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060637269264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060637268880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060631454352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135060631453968: TensorSpec(shape=(), dtype=tf.resource, name=None)
